# Example on how to train a DQN for the created connfect four encironment
This notebook, shows and explains how an AI model can be used to learn how to play the game connect four using reinforcement learning.

## Import packages

In [1]:
import sys
import os
import numpy as np

sys.path.append(os.path.join('..', 'code'))
from connectFour import *
from connectFourGame import *
from agents import *

/Users/basvanderbijl/Library/Mobile Documents/com~apple~CloudDocs/DataGreen/projects/connectFour/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Setup environment

In [2]:
env = ConnectFourGymEnv()

In [3]:
env.observation_space.flatten().shape[0]

42

## Train AI Model

In [4]:
input_shape = env.observation_space.flatten().shape
num_actions = len(env.action_space)

agent = DQNAgent(input_shape, num_actions, 30)

In [5]:
agent.model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 20)                860       
                                                                 
 dense_1 (Dense)             (None, 50)                1050      
                                                                 
 dense_2 (Dense)             (None, 7)                 357       
                                                                 
Total params: 2267 (8.86 KB)
Trainable params: 2267 (8.86 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
episodes = 100
batch_size = 100

rewards = []

for e in range(episodes):
    state = env.reset()
    state = np.reshape(state, (1,-1))
    
    while not env.game_over:
        action = agent.make_move(env.agent.current_player, state, env.action_space)
        new_state, reward, done, info = env.step(action)
        
        if env.agent.current_player == 0:
            agent.memorize(state, action, reward, new_state, done)
        state = new_state
        state = np.reshape(state, (1,-1))
    
    rewards.append(reward)
        
    agent.learn(batch_size)


In [ ]:
agent.save("agentv1.hdf5")